In [ ]:
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./webdriver/chromedriver')

rating_df = pd.DataFrame()
restaurant_df = pd.DataFrame()


# 가게 정보를 담는 클레스
class STORE_Info:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    place_name = ''
    place_address = ''
    place_local  = ''
    place_category  = ''
    
    # 생성자
    def __init__(self,  place_name = None, place_address = None, place_local = None, place_category = None):
        self.place_name =  place_name
        self.place_address = place_address
        self.place_local  = place_local
        self.place_category  = place_category

        
# 리뷰 정보를 담는 클레스
class REVIEW_Info:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    commen = ''
    rating = ''
    user_id  = ''
    timestamp  = ''

    
    # 생성자
    def __init__(self,  comment = None, rating = None, user_id = None, timestamp = None):
        self.comment =  comment
        self.rating = rating
        self.user_id  = user_id
        self.timestamp  = timestamp         

def main():
    global driver, load_wb, review_num, rating_df, restaurant_df

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

     # 검색할 목록
#     place_infos = ['종로구 맛집', '중구 맛집', ' 용산구 맛집', '성동구 맛집', '광진구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '강북구 맛집']
#     place_infos = ['도봉구 맛집', '노원구 맛집', '은평구 맛집', '서대문구 맛집', '마포구 맛집', '양천구 맛집', '강서구 맛집','구로구 맛집']
#     place_infos = ['금천구 맛집', '영등포구 맛집', '동작구 맛집', '관악구 맛집', '서초구 맛집', '강남구 맛집', '송파구 맛집','강동구 맛집']
  
    place_infos = ['금천구 맛집']
    
    for i, place in enumerate(place_infos):
        print("#####", i,":",place)
        search(place)
        
#         rating_df.to_csv('%s_rating_df.csv' %place)
#         rating_df.to_csv('%s_rating_df_ko.csv' %place, sep=',', na_rep='NaN', encoding='utf-8-sig')
        
#         restaurant_df.to_csv('%s_restaurant_df.csv'  %place)
#         restaurant_df.to_csv('%s_restaurant_df_ko.csv'  %place, sep=',', na_rep='NaN', encoding='utf-8-sig')
        
    driver.quit()
    print("finish")


#### 가게 정보 찾기
def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    xPath = '//*[@id="info.search.page.no1"]'
    driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 전체 페이지
    while True:
        try:
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
                crawling(place, place_lists)
                
                # 다음 페이지 넘기기
                if i==5:
                    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

        except ElementNotInteractableException:
            print('end page')
            break
            
        finally:
            search_area.clear()
        

#### 가게 정보 크롤링 하기
def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    
    global restaurant_df

    while_flag = False
    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_local = place.select('.info_item > .addr > .lot_number')[0].text
        place_category = place.select('.head_item > .subcategory')[0].text
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail
        
        STORE_Info(place_name, place_address, place_local, place_category)
        
        #### DataFrame 저장 ####
        row = {"ItemID":place_name, "address": place_address, "local" : place_local, "category": place_category}
        row = pd.DataFrame(row, index=[1])
        restaurant_df = restaurant_df.append(row, ignore_index=True)
        
        #### 데이터 베이스 저장 #####

        
        
        
        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            
            sleep(1)
            extract_review(place_name) # 리뷰 추출
            
            # 그 이후 페이지
            while True:
                idx = 4
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출            
            
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


#### 리뷰 크롤링 하기
def extract_review(place_name):
    global driver, rating_df

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
            timestamp = review.select(' div > span.time_write') #시간정보
            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)                      

            #### 클래스 ####
            review_info = REVIEW_Info()
                
            #### 클래스에 정보 담기 ####
            if len(comment) != 0:
                review_info.comment = comment[0].text
            else:
                review_info.comment = None
            
            if len(rating) != 0:
                review_info.rating = rating[0].text.replace('점', '')
            else:
                review_info.rating = None
            
            if len(user_id) != 0:
                review_info.user_id = user_id[0].get('data-userid')
            else:
                review_info.user_id = None
                
            if len(timestamp) != 0:
                review_info.timestamp = timestamp[0].text
            else:
                review_info.timestamp = None
            
            
            #### DataFrame 에 정보 담기 ####
            try:
                row = {"ItemID": place_name, "UserID":review_info.user_id, "review" : review_info.comment,
                       "Rating":review_info.rating, "Timestamp":review_info.timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row, ignore_index=True)
            
            except:
                row = {"ItemID":place_name, "UserID": None, "review" : None,
                       "Rating":None, "Timestamp":review_info.timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row, ignore_index=True)
                
    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0 : 금천구 맛집
#### 광고만두라
드뎌열은듯? 전화가네 /4
ㅜㅜ 방문했는데 아직 안열었네요ㅠㅠ 맛보고싶었는데ㅜ/4
주변 상인들 말로는 6월달부터 영업하신다고 전달받았는데 아직까지도 영업하지 않으심.. 5월 중순부터 3번은 찾아가본 듯 한데 언제쯤 먹어볼 수 있을지…/3
못먹어봐서 ㅠ 6월10일 어제 갔는데 개인 사정으로 당분간 쉰다고 적혀 있었어요 ㅠ 언제까지 쉬는지는 잘 모름 안적혀 있었음./3
만두로는 최고/5
금천구 맛집 찾다가 들렸는데 김치만두 최고네요~~ 속도 꽉차고 3개 먹으니 너무 배불르네요/5
/5
우연히 예전 방송에 나온거 보고 갑자기 먹고 싶어서 방문했는데 동네 만두집하고 큰 차이는 모르겠어요 ㅠㅠ/3
통만두 맛집/5
만두 맛집이라고해서 따라가서 먹은 집. 김치만두만 먹어서 김치만두국 먹었는데, 김치만두는 정말 별로였음../2
맛은 그럭저럭 평범하고 불친절해요/2
공구단지내 종사자와 방문객들의 오랜 사랑을 받고 있는 집으로 속이 비치는 육향가득 통만두가 맛이 좋다./4
통만두는 맛있지만 김치만두는 최악.만두국도 맛없고 김치만두가 맛있어야 진짜 잘하는집인데, 일반 시장통 만두집에 비해 별차이없는 흔한맛에 공구상가 월세가 비쌀리도 없는데 모든 메뉴가 비싼편이고 만두는 3500원, 만두국은 6천원 받아야 적당한 가격임. /1
김치통만두랑 만둣국 시켰음 매장안은 깔끔하고 만두도 금방 나옴 매장분들 모두 정말 친절하심 김치만두는 평소에 비려서 많이 안좋아하는데 굉장히 부드럽고 공장만두보다 간이 삼삼해서 좋음 만둣국은 사골국물에 물만두가 나오는데  정말정말 부드러워서 후루룩 마셔먹을수 있음 만족만족 대만족 재방문 의사 있음/5
만두피가 얇고 부드러워요. 만두 속도 잘 다져져있어서 피랑 굉장히 잘어울려요. 만둣국이랑 김치만두 먹었는데 둘다 최고!! 만둣국은 사골국물이 진하고 김치만두는 속이 비리지않고 김치맛이 적당히 나서 좋았어요ㅠㅠ 가격도 저렴하고 서비스도 훌륭해요. 만둣국 시키면 밥을 주시는데 리필 가능한 듯 해요!! 많이들 가셨으면